In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import time
import numpy as np
import cv2
from tqdm import tqdm
import tensorflow as tf

import os
import sys
sys.path.append(os.path.join(os.getcwd(), "models"))
sys.path.append(os.path.join(os.getcwd(), "models", "research"))

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

from sort.sort import *

sys.path.append(os.path.join(os.getcwd(), "utils"))
from bb_polygon import *
from img_utils import *
from general_utils import *

import warnings
warnings.filterwarnings('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Preparing object detection model:

[TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md):

1. Right click on the Model name of the model you would like to use;
2. Click on Copy link address to copy the download link of the model;
3. Paste the link in a text editor of your choice. You should observe a link similar to download.tensorflow.org/models/object_detection/tf2/YYYYYYYY/XXXXXXXXX.tar.gz;
4. Copy the XXXXXXXXX part of the link and use it to replace the value of the MODEL_NAME variable in the code shown below;
5. Copy the YYYYYYYY part of the link and use it to replace the value of the MODEL_DATE variable in the code shown below.

In [2]:
MODEL_DATE = '20200711'
MODEL_NAME = 'efficientdet_d3_coco17_tpu-32'

print('Loading model... ', end='')
start_time = time.time()

PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)
PATH_TO_CFG = PATH_TO_MODEL_DIR + "/pipeline.config"
PATH_TO_CKPT = PATH_TO_MODEL_DIR + "/checkpoint"

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model... Done! Took 21.71455979347229 seconds


# Loading COCO labels:

In [3]:
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Loading videos frames and ROI + MOI:

In [4]:
input_dir = './data/videos'
output_dir= './data/frames' 
info_dir = './data/zones-movement_paths'
video_output_dir = './data/videos_with_boundingbox'

video_paths = get_videos(input_dir)
print("Discover {} videos in {}".format(len(video_paths), input_dir))

Discover 2 videos in ./data/videos


# Object detection from video frames:

In [5]:
def video_object_dectection(video_path, frame_output_dir, zone_info_dir, 
                                         video_output_dir, output_to_video = False,
                                         from_frame = 0, to_frame = None, time_stride = 1):   
    extracted_frames = extract_frames_from_video(video_path, frame_output_dir)
    roi, mois = extract_video_info(video_path, zone_info_dir)
    
    img = cv2.imread(extracted_frames[0])
    height, width, layers = img.shape
    size = (width,height)
    
    if output_to_video:
        output_file = os.path.join(video_output_dir, os.path.splitext(os.path.basename(video_path))[0] + '.mp4')
        out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    
    start_time = time.time()
    
    frames_to_look = range(min(from_frame, len(extracted_frames)), 
                           min(to_frame, len(extracted_frames)) if to_frame is not None else len(extracted_frames), 
                           time_stride)    
    for frame_id in tqdm(frames_to_look):    
        frame = extracted_frames[frame_id]
        
        # Get object detection bounding boxes
        frame_img, detections = object_detect_image(frame, detect_fn) 
        
        if output_to_video:
            # draw ROI and bounding boxes onto frame
            image_np_with_detections = draw_roi_on_image(frame_img,roi)
        
            image_np_with_detections = draw_boxes(image_np_with_detections, detections['detection_boxes'], 
                                                  [category_index[i+1]['name'] for i in detections['detection_classes']], detections['detection_scores'],
                                                 max_boxes=100, min_score=0.3)
            out.write(image_np_with_detections[:, :, ::-1])
    
    if output_to_video:
        out.release()
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Done! Took {} seconds for {} frames in video {}'.format(elapsed_time, len(frames_to_look), video_path))

In [6]:
video_object_dectection(video_paths[0], output_dir, info_dir, video_output_dir, True, 0, 50, 1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:55<00:00,  3.51s/it]

Done! Took 175.44084000587463 seconds for 50 frames in video ./data/videos\cam_01.mp4


# Object Detection and Tracking from video frames:

In [7]:
def video_object_dectection_and_tracking(video_path, frame_output_dir, zone_info_dir, 
                                         video_output_dir, output_to_video = False,
                                         from_frame = 0, to_frame = None, time_stride = 1):   
    tracker_1, tracker_2, tracker_3, tracker_4 = Sort(), Sort(), Sort(), Sort()
    track_dict = {}

    extracted_frames = extract_frames_from_video(video_path, frame_output_dir)
    roi, mois = extract_video_info(video_path, zone_info_dir)
    
    img = cv2.imread(extracted_frames[0])
    height, width, layers = img.shape
    size = (width,height)
    
    if output_to_video:
        output_file = os.path.join(video_output_dir, os.path.splitext(os.path.basename(video_path))[0] + '_with_tracking.mp4')
        out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    
    start_time = time.time()
    
    frames_to_look = range(min(from_frame, len(extracted_frames)), 
                           min(to_frame, len(extracted_frames)) if to_frame is not None else len(extracted_frames), 
                           time_stride)    
    for frame_id in tqdm(frames_to_look):    
        frame = extracted_frames[frame_id]
        
        # Get object detection bounding boxes
        frame_img, detections = object_detect_image(frame, detect_fn) 
        
        if output_to_video:
            # draw ROI and bounding boxes onto frame
            image_np_with_detections = draw_roi_on_image(frame_img,roi)
        
        # update SORT trackers 
        min_score = 0.2
        dets_1 = change_detections_to_image_coordinates(detections, roi, width, height, [1,2,4], min_score=min_score)
        if len(dets_1) > 0:
            trackers_1 = tracker_1.update(dets_1)
            update_track_dict(track_dict, trackers_1, frame_id, 1)
            if output_to_video:
                image_np_with_detections = draw_boxes_and_lines(image_np_with_detections, trackers_1, track_dict, 'Type 1')
            
        dets_2 = change_detections_to_image_coordinates(detections, roi, width, height, [3], min_score=min_score)
        if len(dets_2) > 0:
            trackers_2 = tracker_1.update(dets_2)
            update_track_dict(track_dict, trackers_2, frame_id, 2)
            if output_to_video:
                image_np_with_detections = draw_boxes_and_lines(image_np_with_detections, trackers_2, track_dict, 'Type 2')
        
        dets_3 = change_detections_to_image_coordinates(detections, roi, width, height, [6], min_score=min_score)
        if len(dets_3) > 0:
            trackers_3 = tracker_1.update(dets_3)
            update_track_dict(track_dict, trackers_3, frame_id, 3)
            if output_to_video:
                image_np_with_detections = draw_boxes_and_lines(image_np_with_detections, trackers_3, track_dict, 'Type 3')
        
        dets_4 = change_detections_to_image_coordinates(detections, roi, width, height, [8], min_score=min_score)
        if len(dets_4) > 0:
            trackers_4 = tracker_1.update(dets_4)
            update_track_dict(track_dict, trackers_4, frame_id, 4)
            if output_to_video:
                image_np_with_detections = draw_boxes_and_lines(image_np_with_detections, trackers_4, track_dict, 'Type 4')
        
        if output_to_video:
            out.write(image_np_with_detections[:, :, ::-1])
    
    if output_to_video:
        out.release()
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Done! Took {} seconds for {} frames in video {}'.format(elapsed_time, len(frames_to_look), video_path))
    
    return track_dict

In [8]:
tracks = video_object_dectection_and_tracking(video_paths[0], output_dir, info_dir, video_output_dir, True, 0, 50, 1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:49<00:00,  2.18s/it]

Done! Took 109.18736505508423 seconds for 50 frames in video ./data/videos\cam_01.mp4


# TODO: count vehicles